#Leer OCR

In [ ]:
import subprocess

# Instalar Tesseract y el paquete de idioma español
subprocess.run(['sudo', 'apt-get', 'install', '-y', 'tesseract-ocr'], check=True)
subprocess.run(['sudo', 'apt-get', 'install', '-y', 'tesseract-ocr-spa'], check=True)

# Instalar ImageMagick
subprocess.run(['sudo', 'apt-get', 'install', '-y', 'imagemagick'], check=True)

# Instalar las librerías de Python
subprocess.run(['pip', 'install', 'pyocr'], check=True)
subprocess.run(['pip', 'install', 'wand'], check=True)
subprocess.run(['pip', 'install', 'pillow'], check=True)

from google.colab import drive


##Script

In [ ]:
import sys
import pyocr
import pyocr.builders
from wand.image import Image as WandImage
from PIL import Image as PILImage


def show_config():
    tools = pyocr.get_available_tools()
    if len(tools) == 0:
        print("Herramienta OCR no encontrada.")
        sys.exit(1)

    tool = tools[0]
    print("* Herramientas OCR disponibles:")
    for t in tools:
        print('  - %s' % t.get_name())
        langs = t.get_available_languages()
        print("    - Lenguajes disponibles: %s" % ", ".join(langs))

show_config()


* Herramientas OCR disponibles:
  - Tesseract (sh)
    - Lenguajes disponibles: eng, osd, spa
  - Tesseract (C-API)
    - Lenguajes disponibles: eng, osd, spa


In [ ]:
#show_config()

tool = pyocr.get_available_tools()[0]
lang = 'spa'

##GuardarFicheros

In [ ]:
from wand.image import Image as WandImage
from PIL import Image as PILImage
import pyocr
import pyocr.builders
import io

# Asegúrate de que pyocr está configurado correctamente
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("Herramienta OCR no encontrada.")
    sys.exit(1)

tool = tools[0]

# Cargar un archivo de imagen multipágina (por ejemplo, un PDF)
pdf_path = '//content/drive/MyDrive/gii/Copia de Juan_Carlos_Upegui-2098183347208652-1'  # Asegúrate de que esta ruta sea correcta y el archivo exista
with WandImage(filename=pdf_path, resolution=300) as image_pdf:
    image_jpeg = image_pdf.convert('jpeg')

# Guardar todas las imágenes en un array
page_jpeg_list = []
for img in image_jpeg.sequence:
    with WandImage(image=img) as img_page:
        page_jpeg_list.append(PILImage.open(io.BytesIO(img_page.make_blob('jpeg'))))

# Mostrar información sobre las imágenes convertidas
for i, img in enumerate(page_jpeg_list):
    print(f"Página {i + 1}: tamaño {img.size}, modo {img.mode}")

##Extraer la info
page_text_list = []
for img in page_jpeg_list:
    # Convertir la imagen a RGB si no lo está
    img = img.convert("RGB")
    text = tool.image_to_string(img, lang=lang, builder=pyocr.builders.TextBuilder())
    page_text_list.append(text)
    print('- Página %2s: %5s caracteres' % (len(page_text_list), len(text)))

# Guardar el texto en un archivo
file_path = '/content/drive/MyDrive/gii/textfile.txt'
with open(file_path, 'w') as fp:
    fp.write('\n\n'.join(page_text_list))

Página 1: tamaño (600, 475), modo RGB
- Página  1:   153 caracteres


#Identificar tipo de archivo

In [1]:
#Importar librerías
!pip install python-magic
!pip install tqdm
import os
import json
import magic
import requests
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from google.colab import drive

ERROR: Operation cancelled by user


In [ ]:
# Montar Google Drive
drive.mount('/content/drive')

# ruta de archivos
carpeta = "/content/drive/MyDrive/Proyecto_grado/multimedia candidatos/"

#Carga del df
df = pd.read_csv('/content/drive/MyDrive/Proyecto_grado/Transcripciones/Transcripciones.csv', header=0)

# Iterar sobre los archivos en la carpeta
for archivo in tqdm(os.listdir(carpeta)):

    ruta_archivo = os.path.join(carpeta, archivo)

    # Crea un objeto Magic para detectar el tipo de archivo
    detector = magic.Magic(mime=True)

    #id del documento
    id=archivo.split("-")
    id=id[1]
    if "." in id:
      id=id.split(".")
      id=id[0]

    if df.empty:
        print("El DataFrame está vacío.")
    elif not df.loc[df['id'] == int(id) ].empty:
        print('----------------------------')
        print(f"id ya procesada - id: {id}")
        continue

    # Abre el archivo en modo binario
    with open(ruta_archivo, 'rb') as f:
        # Lee los primeros bytes del archivo para determinar su tipo
        primeros_bytes = f.read(1024)  # Lee los primeros 1024 bytes (puedes ajustar este valor según tus necesidades)

        # Utiliza Magic para determinar el tipo de archivo basado en los primeros bytes
        tipo = detector.from_buffer(primeros_bytes)

        if "video" in (tipo):
          print('----------------------------')
          print(archivo+' - tipo: '+tipo)
          print(' procesando video...')

          url = "http://148.251.177.230:8080/upload_video"

          response = requests.post(url,files = {'file': open(ruta_archivo, 'rb')})
          print(str(response.status_code))
          print(str(response.text))

          if response.status_code==200:
            texto= json.loads(response.text)
            texto=texto["text"]

            page_name= archivo.split("-")
            page_name= page_name[0]

            # Crear un DataFrame con la nueva fila
            nueva_fila = pd.DataFrame({"id": [id], "page_name": [page_name], "transcripcion": [texto]})

            # Concatenar el nuevo DataFrame al original
            df = pd.concat([df, nueva_fila], ignore_index=True)

            # guardar resultados
            df.to_csv('/content/drive/MyDrive/Proyecto_grado/Transcripciones/Transcripciones.csv', index=False)

            print('video procesado OK')

        else: # es otro tipo de archivo
          print('----------------------------')
          print('Archivo no procesado - tipo: '+tipo+' - '+archivo)

#Whisper

In [ ]:
import tempfile
import whisper
import traceback
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Aplicar parches para que funcione en Colab
nest_asyncio.apply()

# Crear la app
app = FastAPI()

# Cargar el modelo una vez
model = whisper.load_model("large-v3")

@app.post("/upload_video")
async def upload_video(file: UploadFile = File(...)):
    try:
        print('Procesando archivo...')
        file_content = await file.read()

        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_file:
            temp_file.write(file_content)
            temp_file_path = temp_file.name

        result = model.transcribe(temp_file_path, language='es')

        print(f"Texto transcrito: {result['text']}")
        return JSONResponse(content={"text": result['text']}, status_code=200)

    except Exception as e:
        traceback.print_exc()
        return JSONResponse(content={"error": str(e)}, status_code=500)

@app.get("/")
async def root():
    return {"message": "API de Whisper funcionando en Colab"}

# Exponer con ngrok
public_url = ngrok.connect(8000)
print(f"Tu API está disponible en: {public_url}")

# Iniciar servidor
uvicorn.run(app, host="0.0.0.0", port=8000)

